In [1]:
import os
from utils.dataset import SpectrogramsDataset
from utils.preprocessing import transform
from torch.utils.data import random_split
from torch import Generator

#split dataset deterministically
spec_dir = os.path.join('data', 'spec')
features_path = os.path.join('data', 'features.csv')
dataset = SpectrogramsDataset(spec_dir, features_path, transform, target='danceability')
trainset, valset, testset = random_split(dataset, [0.8, 0.1, 0.1], Generator().manual_seed(42))
len(trainset), len(valset), len(testset)

(759, 95, 94)

In [2]:
from torch.utils.data import DataLoader

#configure batches
kwargs = {
    'batch_size': 4, #TODO tune with Neptune
    'num_workers': 2,
    'pin_memory': True,
    'drop_last': True}
train_loader = DataLoader(trainset, shuffle=True, persistent_workers=True, **kwargs)
val_loader = DataLoader(valset, persistent_workers=True, **kwargs)
test_loader = DataLoader(testset, **kwargs)
next(iter(train_loader))[0].shape

torch.Size([4, 3, 224, 224])

# TODO

* Adjust below
    * [Image classification example](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
    * [Tabular regression example](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)
    * [My VAE](https://github.com/octodoge/dssc-pml/blob/main/code/vae_pyro.ipynb)
    * [Marta's classifier](https://github.com/15Max/MusicalGenreRecognition/blob/main/project.ipynb)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module): #TODO move it in utils/models.py and adjust the imports
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding='same')
        self.fc1 = nn.Linear(16 * 56 * 56, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
import torch.optim as optim

#define model, loss function and optimizer
model = Model()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [5]:
import numpy as np
import torch

#TODO device and move model and batches

#training parameters
n_epochs = 2

#hold the best model
best_mse = np.inf
best_weights = None
history = []

#training loop
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        # forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
        # print progress
        print(loss)
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())

# restore model and return best accuracy
model.load_state_dict(best_weights)

/home/mathias/miniconda3/envs/spectrofy/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.4064, grad_fn=<MseLossBackward0>)
tensor(0.2810, grad_fn=<MseLossBackward0>)
tensor(0.1326, grad_fn=<MseLossBackward0>)
tensor(0.1160, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 